# Module 2 Final Project Submission

Student name: Fernando Aguilar Islas<br>
Student pace: Part Time<br>
Scheduled project review date/time: TBA<br>
Instructor name: Brandon Lewis<br>
Blog post URL:<br>
Video Walkthrough URL:<br>

## 1. Introduction
For this project I will solve relevant questions utilizing hypothesis testing and statistical analysis using information from the Northwind Database, published by Micorsoft. The tables contain company's information about operating regions, suppliers, employees, clients, orders, and products. Below is the Entity Relationship Diagram (ERD) of the database I will be working with.<br>
### Northwind's SQL Database

<img src='Northwind_ERD.png'>

## 2. Objective <br>

The priority is to find startegies that will increase Northiwind's profits, hence, I will be looking into solutions that either increase revenue or cut down costs. The following questions will be answered thoruought the notebook, each with its own section stating its null and alternate hypothesis, and carrying out statistical testing to verify the significance of the findings. This will enable me to provide Northwind with relevant recommendations in how to improve their operations to obtain a higher profit.<br>

1. **Do discounts have a statistically significant effect on the number of products customers order? If so, at what level(s) of discount?**
2. **Do different regions have different employee productivity?**
3. **Another good question?**
4. **Another great question?**
5. **The greatest question?**<br>

Greatest


## 3. Methodology

In [1]:
#Import sql libraries to retrieve, manipulate and explore data from database.
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import inspect
import sqlite3 as sql

#Import tools for Exploratory Data Analysis (EDA)
import pandas as pd
import numpy as np

#Import tools for data visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#Import tools for hypothesis testing
from scipy import stats
import itertools
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import MinMaxScaler

#Helper functions for presentation purposes
from ipywidgets import interact, interactive
import warnings
warnings.filterwarnings('ignore')

In [2]:
engine = create_engine("sqlite:///Northwind_small.sqlite", echo=True)
Session = sessionmaker(bind=engine)
session = Session()
inspector = inspect(engine)

2019-06-03 17:10:45,910 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-06-03 17:10:45,911 INFO sqlalchemy.engine.base.Engine ()
2019-06-03 17:10:45,912 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-06-03 17:10:45,912 INFO sqlalchemy.engine.base.Engine ()


In [3]:
# Obtain and store the names of tables in database as as list.
db_tables=inspector.get_table_names()
db_tables

2019-06-03 17:10:45,917 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-06-03 17:10:45,918 INFO sqlalchemy.engine.base.Engine ()


['Category',
 'Customer',
 'CustomerCustomerDemo',
 'CustomerDemographic',
 'Employee',
 'EmployeeTerritory',
 'Order',
 'OrderDetail',
 'Product',
 'Region',
 'Shipper',
 'Supplier',
 'Territory']

## 4.  Do discounts have a statistically significant effect on the number of products customers order? If so, at what level(s) of discount?<br>
> $H_{0}:$ *Discounts do not have an effect on the number of products ordered.* <br> 
> $H_{1}:$ *Discounts have an effect on the number of products ordered.* <br>

The relevant tables that cotain the information needed to answer this question are Order and OrderDetail. I will start by creating a merged dataframe 

In [8]:
df_orderd = pd.read_sql_query("SELECT * FROM [OrderDetail]", engine)
df_orderd.head()

2019-06-03 17:11:27,089 INFO sqlalchemy.engine.base.Engine SELECT * FROM [OrderDetail]
2019-06-03 17:11:27,090 INFO sqlalchemy.engine.base.Engine ()


,Id,OrderId,ProductId,UnitPrice,Quantity,Discount
0,10248/11,10248,11,14.0,12,0.0
1,10248/42,10248,42,9.8,10,0.0
2,10248/72,10248,72,34.8,5,0.0
3,10249/14,10249,14,18.6,9,0.0
4,10249/51,10249,51,42.4,40,0.0


In [9]:
df_orders = df_order.set_index('Id').join(df_orderd.set_index('OrderId'))

In [10]:
df_orders.head()

,CustomerId,EmployeeId,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry,Id,ProductId,UnitPrice,Quantity,Discount
10248,VINET,5,2012-07-04,2012-08-01,2012-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,Western Europe,51100,France,10248/11,11,14.0,12,0.0
10248,VINET,5,2012-07-04,2012-08-01,2012-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,Western Europe,51100,France,10248/42,42,9.8,10,0.0
10248,VINET,5,2012-07-04,2012-08-01,2012-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,Western Europe,51100,France,10248/72,72,34.8,5,0.0
10249,TOMSP,6,2012-07-05,2012-08-16,2012-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,Western Europe,44087,Germany,10249/14,14,18.6,9,0.0
10249,TOMSP,6,2012-07-05,2012-08-16,2012-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,Western Europe,44087,Germany,10249/51,51,42.4,40,0.0


## 5.  Another question?<br>

In [7]:
df_order = pd.read_sql_query("SELECT * FROM [Order]", engine)
df_order.head()

2019-06-03 17:11:23,152 INFO sqlalchemy.engine.base.Engine SELECT * FROM [Order]
2019-06-03 17:11:23,153 INFO sqlalchemy.engine.base.Engine ()


,Id,CustomerId,EmployeeId,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10248,VINET,5,2012-07-04,2012-08-01,2012-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,Western Europe,51100,France
1,10249,TOMSP,6,2012-07-05,2012-08-16,2012-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,Western Europe,44087,Germany
2,10250,HANAR,4,2012-07-08,2012-08-05,2012-07-12,2,65.83,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,South America,05454-876,Brazil
3,10251,VICTE,3,2012-07-08,2012-08-05,2012-07-15,1,41.34,Victuailles en stock,"2, rue du Commerce",Lyon,Western Europe,69004,France
4,10252,SUPRD,4,2012-07-09,2012-08-06,2012-07-11,2,51.30,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,Western Europe,B-6000,Belgium
